In [12]:
import pandas as pd
import numpy as np
import os
import sys
sys.path.append("../code/data_processing/name_resolution/")
from resolved_names_curator import ResolvedNamesCurator

In [36]:
unresolved_names_path=f"./data/name_resolution/unresolved_plant_names.csv"
resolved_names_path=f"./data/name_resolution/taxonome/tpl/resolved_plant_names.csv"
complementary_resolved_names_path = f"./data/name_resolution/taxonome/wfo/resolved_plant_names.csv"

unresolved_names = pd.read_csv(unresolved_names_path)
resolved_names = pd.read_csv(resolved_names_path).rename(columns={"Coded Name": "resolved_name",
                                                                  "Original name": "original_name",
                                                                  "Matched Name": "matched_name",
                                                                  "Coded Authority": "authority"})
resolved_names.resolved_name = resolved_names.resolved_name.apply(lambda name: name.replace("_", " ") if pd.notna(name) else np.nan)
resolved_names.loc[resolved_names.resolved_name.notna(), "matched_name_wo_authority"] = resolved_names.loc[resolved_names.resolved_name.notna()][["matched_name", "authority"]].apply(lambda record: record.matched_name.replace(f" {record.authority}", "").replace(" None",""), axis=1)

In [39]:
complementary_resolved_names = pd.read_csv(complementary_resolved_names_path).rename(columns={"Coded Name": "resolved_name",
                                                                                              "Original name": "original_name",
                                                                                              "Matched Name": "matched_name",
                                                                                              "Coded Authority": "authority"})
complementary_resolved_names.resolved_name = complementary_resolved_names.resolved_name.apply(lambda name: name.replace("_", " ") if pd.notna(name) else np.nan)

In [31]:
resolved_names.set_index("original_name", inplace=True)
complementary_resolved_names.set_index("original_name", inplace=True)
for c in complementary_resolved_names.columns:
    resolved_names[c].fillna(value=complementary_resolved_names[c].to_dict(), inplace=True)
resolved_names.reset_index(inplace=True)
complementary_resolved_names.reset_index(inplace=True)

In [32]:
print(f"# unresovled names = {unresolved_names.shape[0]:,}")
print(f"# resolved names = {len(resolved_names.resolved_name.unique()):,}")
print(f"% coverage by name resolution = {np.round(resolved_names.query('resolved_name.notna()').shape[0]/unresolved_names.shape[0]*100,2)}%")
print(f"# resolved names that were declared as synonyms = {len(resolved_names.loc[resolved_names.matched_name_wo_authority != resolved_names.resolved_name].resolved_name.unique()):,}")

# unresovled names = 5,310
# resolved names = 4,007
% coverage by name resolution = 77.82%
# resolved names that were declared as synonyms = 1,289


In [33]:
genus_names = resolved_names.loc[~resolved_names.resolved_name.str.contains(" ", na=False), "resolved_name"].tolist()
print(f"# names resolved at genus level = {len(genus_names):,}")

# names resolved at genus level = 2,043


In [34]:
missing_names = resolved_names.query("resolved_name.isna()").original_name.tolist()
print(f"# missing names = {len(missing_names):,}")

# missing names = 1,115


In [35]:
resolved_names_to_doc = resolved_names[["original_name", "matched_name", "resolved_name"]]
resolved_names_to_doc.original_name = resolved_names_to_doc.original_name.str.lower()
resolved_names_to_doc.matched_name = resolved_names_to_doc.matched_name.str.lower()
resolved_names_to_doc.resolved_name = resolved_names_to_doc.resolved_name.str.lower()
resolved_names_to_doc.to_csv(f"./data/name_resolution/resolved_plant_names.csv", index=False)

/tmp/ipykernel_15289/2840792341.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resolved_names_to_doc.original_name = resolved_names_to_doc.original_name.str.lower()
/tmp/ipykernel_15289/2840792341.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resolved_names_to_doc.matched_name = resolved_names_to_doc.matched_name.str.lower()
/tmp/ipykernel_15289/2840792341.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va